In [ ]:
import os
from crewai import Agent, Task, Crew

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "OPENAI_API_KEY not set. "
        "Add it as a GitHub Codespaces secret or local env variable."
    )

print("✅ Environment ready")

In [ ]:
from crewai import LLM

llm = LLM(
    model="gpt-5-mini",
    temperature=1
)

print(f"✅ Using model: {llm.model}")

In [ ]:
from crewai import Agent

writer = Agent(
    role="LinkedIn Post Writer",
    goal="Write engaging, thoughtful LinkedIn posts on a given topic",
    backstory=(
        "You are an experienced professional writer who understands "
        "LinkedIn tone: insightful, concise, and authentic. "
        "You focus on clarity and value, not hype."
    ),
    llm=llm,
    verbose=True
)

print("✅ Writer agent ready")

In [ ]:
MAX_LINES = int(input("Max number of lines? (e.g. 2): "))
USE_EMOJIS = input("Use emojis? (y/n): ").lower() == "y"
TOPIC = input("What is the LinkedIn post about? ")

print(f"✅ Constraints: max_lines={MAX_LINES}, emojis={USE_EMOJIS}")
print(f"✅ Topic: {TOPIC}")

In [ ]:
from crewai import Agent

emoji_rule = (
    "You MAY use relevant emojis sparingly."
    if USE_EMOJIS
    else "Do NOT use emojis."
)

editor = Agent(
    role="LinkedIn Editor",
    goal=(
        f"Edit and compress LinkedIn posts into a maximum of {MAX_LINES} lines. "
        f"{emoji_rule}"
    ),
    backstory=(
        "You are a ruthless LinkedIn editor. "
        "You enforce constraints strictly, remove fluff, "
        "and optimize for clarity, punch, and readability."
    ),
    llm=llm,
    verbose=True
)

print("✅ Editor agent ready")


In [ ]:
from crewai import Task, Crew

approved = False
feedback = ""

while not approved:
    # Task 1: Content generation
    write_task = Task(
        description=(
            f"Write a full LinkedIn post about: {TOPIC}.\n"
            "Focus on insight, clarity, and value.\n"
            f"User feedback to incorporate (if any): {feedback}"
        ),
        expected_output=(
            "A complete LinkedIn post with a strong hook, clear structure, "
            "and thoughtful insight."
        ),
        agent=writer
    )

    # Task 2: Editing / constraint enforcement
    edit_task = Task(
        description=(
            f"Edit the writer's post into a final LinkedIn post.\n"
            f"- Maximum {MAX_LINES} lines\n"
            f"- {'Emojis allowed' if USE_EMOJIS else 'NO emojis'}\n"
            "Be concise, punchy, and readable."
        ),
        expected_output=(
            f"A polished LinkedIn post of no more than {MAX_LINES} lines, "
            f"{'with' if USE_EMOJIS else 'without'} emojis."
        ),
        agent=editor
    )

    crew = Crew(
        agents=[writer, editor],
        tasks=[write_task, edit_task],
        verbose=True
    )

    result = crew.kickoff()

    print("\n================ FINAL DRAFT ================\n")
    print(result)
    print("\n============================================\n")

    decision = input("Approve this post? (y/n): ").strip().lower()
    if decision == "y":
        approved = True
        print("✅ Approved.")
    else:
        feedback = input("What should change? ").strip()
        print("🔁 Regenerating with feedback...\n")
